In [47]:
""" For the word prompts dataset, detokenizing and preparing the words to be input into GPT2"""


# the only thing that needs to be detokenized is punctuation being separated by a space. And there being newline characters. (should these be replaced??)

from sacremoses import MosesTokenizer, MosesDetokenizer
import pandas as pd
md = MosesDetokenizer(lang='en')

for variant in ['test']:#, 'train']:
    cleaned = dict()
    
    for s_or_t in ['source', 'target']:
        f = variant+'.wp_'+s_or_t
    
        with open(f, 'r') as file:
            text = file.readlines()
        detokened = []
        for ind in range(0,len(text)):
            detokened.append(md.detokenize(text[ind:ind+1]).replace('<newline>', '\n'))
        assert len(text)==len(detokened), "text and detokened are not the same length!"
        cleaned[variant+'_'+s_or_t] = detokened
        
    pd.DataFrame.from_dict(cleaned).to_csv(variant+'_dataframe.csv', index=False)


# need to find some way to keep things that are the right token length. 

In [2]:
import pandas as pd
df = pd.DataFrame.from_dict(cleaned)

NameError: name 'cleaned' is not defined

In [66]:
df.head()

,test_source,test_target
0,[WP] Leonardo DiCaprio in a fit of rage begins...,The wet marble floor pressed on his cheek like...
1,[CW] Kill the writer in first-person narrative.,It's been three days since my boyfriend pissed...
2,[EU] Sean Bean has a hard time leaving his rol...,"George fled. Not terrifically fast, not at his..."
3,[WP] A kid doodling in a math class accidental...,It was dark and Levi was pretty sure he was ly...
4,[WP] As a Space marine you have an allowance o...,``Hi dad! ''\n \n The words snap me from my th...


In [109]:
df.shape

(15138, 4)

In [108]:
# how many unique prompts are there? 
df['test_source'].unique().shape

(5478,)

In [91]:
'[' in '[WP]'

True

In [112]:
'''starters = []
for e in df['test_source'].str[:30].unique().tolist():
    if '[' in e:
        starters.append(e)
starts = []
for s in starters:
    starts.append(s.split(']')[0][1:])
list(set(starts))'''

"starters = []\nfor e in df['test_source'].str[:30].unique().tolist():\n    if '[' in e:\n        starters.append(e)\nstarts = []\nfor s in starters:\n    starts.append(s.split(']')[0][1:])\nlist(set(starts))"

In [114]:
'''df['test_source'][df['test_source'].str.contains('Wiring Promp')]'''

1748     [Wiring Prompt] A person who commits a crime f...
7691     [Wiring Prompt] A person who commits a crime f...
11362    [Wiring Prompt] A person who commits a crime f...
13895    [Wiring Prompt] A person who commits a crime f...
14969    [Wiring Prompt] A person who commits a crime f...
Name: test_source, dtype: object

In [139]:
import numpy as np
df['Prompt'] = df['test_source'] + " \n "+df['test_target'].apply(lambda x: ' '.join(x.split(' ')[:300]))
df.head()

,test_source,test_target,test_target_200,Prompt
0,[WP] Leonardo DiCaprio in a fit of rage begins...,The wet marble floor pressed on his cheek like...,The wet marble floor pressed on his cheek like...,[WP] Leonardo DiCaprio in a fit of rage begins...
1,[CW] Kill the writer in first-person narrative.,It's been three days since my boyfriend pissed...,It's been three days since my boyfriend pissed...,[CW] Kill the writer in first-person narrative...
2,[EU] Sean Bean has a hard time leaving his rol...,"George fled. Not terrifically fast, not at his...","George fled. Not terrifically fast, not at his...",[EU] Sean Bean has a hard time leaving his rol...
3,[WP] A kid doodling in a math class accidental...,It was dark and Levi was pretty sure he was ly...,It was dark and Levi was pretty sure he was ly...,[WP] A kid doodling in a math class accidental...
4,[WP] As a Space marine you have an allowance o...,``Hi dad! ''\n \n The words snap me from my th...,``Hi dad! ''\n \n The words snap me from my th...,[WP] As a Space marine you have an allowance o...


In [140]:
df['Prompt'] = df['Prompt'].apply(lambda x: x.replace('`` ', ' ``'))

In [141]:
pd.options.display.max_rows = 999
df['Prompt'].head(100)

0     [WP] Leonardo DiCaprio in a fit of rage begins...
1     [CW] Kill the writer in first-person narrative...
2     [EU] Sean Bean has a hard time leaving his rol...
3     [WP] A kid doodling in a math class accidental...
4     [WP] As a Space marine you have an allowance o...
5     [TT] ``Shut the dog up. '' \n “Shut the dog up...
6     [WP] A man finally discovers his superpower......
7     [WP] Reddit buys the moon \n Dickbutt... That ...
8     [WP] You have a very shitty type of precogniti...
9     [WP] ``Before we allow humankind to live among...
10    [WP] A magical mirror shows your reflection an...
11    [WP] A person with a high school education get...
12    [WP] Everyone has a reaper. The further away i...
13    [WP] A person with a high school education get...
14    [WP] After brushing your teeth in the morning ...
15    [WP] While searching the site of a disaster, y...
16    [WP] While walking home from work one day, som...
17    [WP] Fabricate an inspiring encounter with

In [142]:
df.to_csv(variant+'_dataframe_300primer.csv', index=False)

# Load in and processes what was already there! 

In [4]:
import pandas as pd
df=pd.read_csv('test_dataframe_500primer.csv')
df.loc[5, 'Prompt']

"[TT] ``Shut the dog up. '' \n “Shut the dog up, ” shouted my head officer from the jeep. The dog was running circles around our vehicle, barking at the people inside. The officer tapped my shoulder and pointed to the yellow, skinny animal circling our jeep. \n \n“ But sir.., ” I managed to spit out before he took both his hands and pushed me out of the vehicle. I went tumbling out, and landed on the rough sandy ground. I stood up adjusting the gun hanging from my shoulder and proceeded to walk towards the canine. The dog stopped its barking, and shifted its black eyes to me. \n \n “Come here little pup. Hey come here, I ’ m not going to hurt ya, ” I said trying to coax it nearer to me. Actually, I didn ’ t know if I was going to hurt the little mutt or not yet. Reaching my hand towards my waist, I pulled off a tiny bit of my rations. I held it out my hand, with the ration laying on my open palm. The dog perked it ’ s ears, and came a few inches closer to me. Ever so slightly the skitt

In [5]:
df.head()

,test_source,test_target,Prompt
0,[WP] Leonardo DiCaprio in a fit of rage begins...,The wet marble floor pressed on his cheek like...,[WP] Leonardo DiCaprio in a fit of rage begins...
1,[CW] Kill the writer in first-person narrative.,It's been three days since my boyfriend pissed...,[CW] Kill the writer in first-person narrative...
2,[EU] Sean Bean has a hard time leaving his rol...,"George fled. Not terrifically fast, not at his...",[EU] Sean Bean has a hard time leaving his rol...
3,[WP] A kid doodling in a math class accidental...,It was dark and Levi was pretty sure he was ly...,[WP] A kid doodling in a math class accidental...
4,[WP] As a Space marine you have an allowance o...,``Hi dad! ''\n \n The words snap me from my th...,[WP] As a Space marine you have an allowance o...


In [9]:
df.iloc[0].Prompt

"[WP] Leonardo DiCaprio in a fit of rage begins to torpedo his own career by deliberately acting poorly and taking on bad films. He finally wins an oscar for starring in Paul Blart: Mall Cop 3. \n The wet marble floor pressed on his cheek like a thousand hands slapping his face frozen in time. Smattering piss of rain ignored his indignant mumblings. His eyes fluttered. Pins and needs ran from finger to shoulder as he pushed back against the floor, contorting his aching body into a cross legged position. Last night was bad. He gathered that. His routine dullness of though crept inwards from the edges of his mind toward the black mist that veiled his most recent memories. He struggled to recall whatever he could n't recall but only for a moment before he decided it probably was n't worth the effort. \n He glanced around the room for a few minutes before concluding that he probably did n't know where he was. His investigation was n't entirely fruitless, he discovered a mostly full bottle 

In [6]:
import numpy as np
df['Prompt'] = df['test_source'] + " \n "+df['test_target'].apply(lambda x: ' '.join(x.split(' ')[:500]))
df['Prompt'] = df['Prompt'].apply(lambda x: x.replace('`` ', ' ``'))
df.head()

,test_source,test_target,test_target_200,Prompt
0,[WP] Leonardo DiCaprio in a fit of rage begins...,The wet marble floor pressed on his cheek like...,The wet marble floor pressed on his cheek like...,[WP] Leonardo DiCaprio in a fit of rage begins...
1,[CW] Kill the writer in first-person narrative.,It's been three days since my boyfriend pissed...,It's been three days since my boyfriend pissed...,[CW] Kill the writer in first-person narrative...
2,[EU] Sean Bean has a hard time leaving his rol...,"George fled. Not terrifically fast, not at his...","George fled. Not terrifically fast, not at his...",[EU] Sean Bean has a hard time leaving his rol...
3,[WP] A kid doodling in a math class accidental...,It was dark and Levi was pretty sure he was ly...,It was dark and Levi was pretty sure he was ly...,[WP] A kid doodling in a math class accidental...
4,[WP] As a Space marine you have an allowance o...,``Hi dad! ''\n \n The words snap me from my th...,``Hi dad! ''\n \n The words snap me from my th...,[WP] As a Space marine you have an allowance o...


In [10]:
df.drop(['test_source', 'test_target', 'test_target_200'], axis=1).to_csv('test_dataframe_500primer_only.csv', index=False)

In [11]:
df.drop(['test_target_200'], axis=1).to_csv('test_dataframe_500primer.csv', index=False)

# Getting the ground truth completion

In [9]:
import numpy as np 
import pandas as pd
prompts=pd.read_csv('test_dataframe_500primer.csv')

In [10]:
prompts.head()

,test_source,test_target,Prompt
0,[WP] Leonardo DiCaprio in a fit of rage begins...,The wet marble floor pressed on his cheek like...,[WP] Leonardo DiCaprio in a fit of rage begins...
1,[CW] Kill the writer in first-person narrative.,It's been three days since my boyfriend pissed...,[CW] Kill the writer in first-person narrative...
2,[EU] Sean Bean has a hard time leaving his rol...,"George fled. Not terrifically fast, not at his...",[EU] Sean Bean has a hard time leaving his rol...
3,[WP] A kid doodling in a math class accidental...,It was dark and Levi was pretty sure he was ly...,[WP] A kid doodling in a math class accidental...
4,[WP] As a Space marine you have an allowance o...,``Hi dad! ''\n \n The words snap me from my th...,[WP] As a Space marine you have an allowance o...


In [ ]:
prompts['Prompt And Completion'] = prompts['test_source'] + " \n " + prompts['test_target'].apply(lambda x: x)

In [8]:
len(prompts.loc[0,'Prompt'])

2893

In [ ]:
prompt_and_completion = []
#prompt_length = 100
#generation_length = 150+200 # just for safety
for batch in text:
    for entry in batch:
        prompt_and_completion = decoder_text(entry)
        
gs = pd.DataFrame({'Prompt And Completion':prompt_and_completion})

In [ ]:
gs.head()

### Making the human prompt completions so that the perplexity can be scored

In [ ]:
gs.to_csv('Human_StoryPrompts_Completion.csv', index=False, columns =['Prompt And Completion'])